In [2]:
%pip install -U xpander-sdk
%pip install -qU openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.9 MB/s eta 0:00:00-:--:--
  Attempting uninstall: xpander-sdk
    Found existing installation: xpander-sdk 1.19.0
    Uninstalling xpander-sdk-1.19.0:
      Successfully uninstalled xpander-sdk-1.19.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from xpander_sdk import XpanderClient, LLMProvider, FriendliAISupportedModels
from openai import OpenAI
from dotenv import load_dotenv
import os

## Load environment variables
load_dotenv()
friendli_client = OpenAI(
    base_url="https://api.friendli.ai/serverless/v1",
    api_key=os.environ.get("FRIENDLI_TOKEN")
)

xpanderAPIKey = os.environ.get("XPANDER_API_KEY","")
xpanderAgentID = os.environ.get("XPANDER_AGENT_ID", "")


xpander_client = XpanderClient(api_key=xpanderAPIKey)
xpander_agent = xpander_client.agents.get(agent_id=xpanderAgentID)


In [3]:
# verify the friendli client is working
completion = friendli_client.chat.completions.create(
    model="meta-llama-3.1-8b-instruct",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='Hello. How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [4]:
print(xpander_agent.get_tools())

[{'type': 'function', 'function': {'name': 'PgTavilyFetchInsightsFromTavilyAI', 'description': 'This operation fetches insights from Tavily AI based on user-defined queries. Use this operation when you need to gather comprehensive information or insights on a specific topic, making it ideal for research, content creation, or data analysis. It is particularly useful for applications that require in-depth exploration of subjects, allowing users to investigate, study, or analyze topics thoroughly.', 'parameters': {'type': 'object', 'properties': {'queryParams': {'type': 'object', 'properties': {}, 'required': []}, 'pathParams': {'type': 'object', 'properties': {}, 'required': []}, 'bodyParams': {'type': 'object', 'properties': {'inputTask': {'type': 'string', 'description': 'input user sub task based on the input query'}}, 'required': []}}, 'required': ['query_params', 'path_params', 'body_params']}}}]


In [7]:
memory = [{"role": "system", "content": "You are a helpful assistant with function calling and tool access. you are running in While loop if you want to stop the loop please add ##FINAL ANSWER## in your answer"},
          {"role": "user", "content": "creat report on AWS Nova new model"}
          ]
print(memory[-1])

number_of_calls = 1
while True:
    llm_response = friendli_client.chat.completions.create(
        model= FriendliAISupportedModels.META_LLAMA_3_1_8_B_INSTRUCT,
        messages=memory,
        tools=xpander_agent.get_tools(),
        tool_choice="auto"
    )
    memory.append({"role": "assistant", "content": f'Step number: {number_of_calls}'})
    print(memory[-1])

    memory.append(llm_response.choices[0].message)
    print(memory[-1])
    model_response = llm_response.choices[0].message
    if model_response.tool_calls:
        print(model_response.tool_calls)
        tools_to_run = XpanderClient.extract_tool_calls(llm_response=llm_response.model_dump(), llm_provider=LLMProvider.FRIENDLI_AI)
        print("Tool Name:", tools_to_run[0].name)
        tool_responses = xpander_agent.run_tools(tool_calls=tools_to_run)
        for tool_response in tool_responses:
            memory.append({"role": "tool", "content": tool_response.result, "tool_call_id": tool_response.tool_call_id})
            print(memory[-1])
    if (model_response.content):
        if "##FINAL ANSWER##" in model_response.content:
            break
    number_of_calls += 1
print(model_response.content)   

{'role': 'user', 'content': 'creat report on AWS Nova new model'}
{'role': 'assistant', 'content': 'Step number: 1'}
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_wTblm7PqOCPC5oktDVBIrm2A', function=Function(arguments='{"queryParams": {}, "pathParams": {}, "bodyParams": {"inputTask": "AWS Nova new model"}}', name='PgTavilyFetchInsightsFromTavilyAI'), type='function')])
[ChatCompletionMessageToolCall(id='call_wTblm7PqOCPC5oktDVBIrm2A', function=Function(arguments='{"queryParams": {}, "pathParams": {}, "bodyParams": {"inputTask": "AWS Nova new model"}}', name='PgTavilyFetchInsightsFromTavilyAI'), type='function')]
Tool Name: PgTavilyFetchInsightsFromTavilyAI
{'role': 'tool', 'content': "system message: graph prompt group selected, ignore this and proceed with the user's request using new tools.", 'tool_call_id': 'call_wTblm7PqOCPC5oktDVBIrm2A'}
{'role': 'assistant', 'content': 'Step n